# Paper Implementation: An Improved Collaborative Filtering Recommendation Algorithm and Recommendation Strategy

This project is based on the paper “An Improved Collaborative Filtering Recommendation Algorithm and Recommendation Strategy” by Xiaofeng Li and Dong Li ​. All research rights and intellectual property belong to the original authors under the Creative Commons Attribution License. We—Matteo and Julian, students at the University of Bolzano—have chosen this work as the foundation for a full analysis and software implementation of its proposed methods, in order to both validate and extend its contributions to community‑aware collaborative filtering.

## 1. Introduction

Li & Li (2019) address key limitations of traditional collaborative filtering (CF)—data sparsity, cold start, and scalability—by integrating overlapping community detection into the CF pipeline. They propose two algorithms to mine user communities from a social network projection of user–item interactions (central‑node‐based and k‑faction). By localizing neighbor selection within these communities and combining rating‐based similarity with category‐based similarity, they demonstrate significant reductions in MAE and RMSE on MovieLens‑100K.

## 2. Implementation Roadmap

Below is our high‑level plan to reproduce and extend Li & Li’s community‑aware CF framework:

1. **Dataset Preparation**
   - Download and preprocess the MovieLens 100K dataset.
   - Build the user–item rating matrix.

2. **Community Detection**
   1. **Central‑Node Algorithm**
      - Compute node degrees; seed each community with the highest‑degree node.
      - Iteratively add neighbors that maximize the local contribution \(q\).
      - Merge any two communities whose overlap \(S \ge 0.7\).
   2. **k‑Faction Algorithm**
      - Use Bron–Kerbosch to extract all cliques of size ≥ *k*.
      - Merge cliques based on an overlap threshold \(T\) and inter‑community connectivity.
      - Assign remaining nodes to their closest community; refine by maximizing modularity \(Q_c\).

3. **Community‑Based Collaborative Filtering**
   - For each target user, restrict neighbor search to their detected community.
   - Construct a user–category binary matrix (e.g. item genres or tags).
   - Compute hybrid similarity:
     \[
       \text{sim}(u,v) = (1 - \lambda)\,\text{sim}_R(u,v) + \lambda\,\text{sim}_{\text{cate}}(u,v).
     \]
   - Predict ratings by aggregating the top‑*K* most similar users’ ratings.

4. **Evaluation Framework**
   - Perform 5‑fold cross‑validation with varying training:test splits (20–80 %).
   - Measure MAE and RMSE for:
     - **CFCD** (Community‑based CF)
     - **CFC** (Cosine CF)
     - **CFP** (Pearson CF)

5. **Parameter Tuning & Experiments**
   - **Experiment 1:** Fix *K* = 30; vary train:test ratio → assess sparsity impact.
   - **Experiment 2:** Fix training ratio at 80 %; vary *K* → find optimal neighbor set size.

6. **Optimizations & Extensions**
   - Scale community detection to large graphs (e.g. using NetworkX/igraph).
   - Incorporate implicit feedback (timestamps, clicks).
   - Prototype a real‑time recommendation pipeline with incremental updates.
   - Explore deep‑learning–based community embeddings as an alternative to classic detection.

7. **Documentation & Reporting**
   - Write clear API docs and usage examples for each module.
   - Produce reproducible scripts and Jupyter notebooks.
   - Summarize results with tables, charts, and a discussion of future work.

In [16]:
%%capture
%pip install pandas numpy surprise scikit-learn networkx matplotlib

In [17]:
import pandas as pd
import networkx as nx
import matplotlib as plt

## Step 1: Dataset Preparation

In this first step we download and preprocess the MovieLens 100K dataset, originally collected by the GroupLens Research Project at the University of Minnesota. The data consists of 100 000 integer ratings (1–5) from 943 users on 1 682 movies (each user has rated at least 20 titles), along with simple demographic information (age, gender, occupation, ZIP code) and detailed movie–genre tags. The raw ratings are stored in a tab‑separated file `u.data` (`user_id | item_id | rating | timestamp`), and a README (`u.info`) describes the number of users, items, and ratings. All research and usage conditions (acknowledgment, non‑commercial use, citation requirements) are documented in the accompanying LICENSE and CITATION sections.

Our code will:
1. Read `u.data` into a DataFrame.
2. Display a sample of raw ratings to verify successful loading.
3. Pivot the user–item ratings into a matrix of shape (943 × 1682), filling missing entries with 0 to prepare for collaborative‑filtering algorithms.


In [18]:
# Step 1: Load and preprocess MovieLens 100K ratings
data_path = './Data/ml-100k/u.data'
columns = ['user_id', 'item_id', 'rating', 'timestamp']

# Read raw ratings file
ratings = pd.read_csv(data_path, sep='\t', names=columns)

# Show a preview of the raw ratings
print("Raw ratings sample:")
display(ratings.head())

# Build user-item rating matrix
rating_matrix = ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Display a small part of the rating matrix
print("\nUser-Item Rating Matrix (first 5 users, first 5 items):")
display(rating_matrix.iloc[:5, :5])

Raw ratings sample:


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596



User-Item Rating Matrix (first 5 users, first 5 items):


item_id,1,2,3,4,5
user_id,,,,,
1,5.0,3.0,4.0,3.0,3.0
2,4.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0


## Step 2: Community Detection Algorithms

### Step 2.1: Central‑Node Overlapping Community Detection Algorithm

In this step we extract overlapping user communities from the user–item interaction graph by seeding each community with the unlabeled node of highest degree (“central node”), then growing it by iteratively adding the neighbor that maximizes the local contribution
`q = Lin / (Lin + Lout)`
(where `Lin` is the count of edges inside the candidate community and `Lout` is the count of edges from that community to the rest of the graph). Expansion stops when no neighbor can increase `q` (the global contribution `Q` is the highest `q` found). Finally, any two communities whose overlap
`S(Ci, Cj) = |Ci ∩ Cj| / |Ci ∪ Cj|`
exceeds 0.7 are merged, repeating until stable. The output is a set of densely connected, overlapping communities to be used as localized neighbor pools in our collaborative‑filtering stage.



In [71]:

def local_contribution(G, C, node):
    """Calculate local contribution q for adding node to community C."""
    C_union = C | {node}
    subgraph = G.subgraph(C_union)
    Lin = subgraph.number_of_edges()
    Lout = sum(1 for u in C_union for v in G.neighbors(u) if v not in C_union)
    return Lin / (Lin + Lout) if (Lin + Lout) > 0 else 0

def community_mining(G):
    """Stage 1: Detect initial overlapping communities based on central nodes."""
    labeled = set()
    communities = []

    while len(labeled) < G.number_of_nodes():
        # Pick highest-degree unlabeled node as seed
        candidates = [n for n in G.nodes if n not in labeled]
        seed = max(candidates, key=G.degree)
        C = {seed}
        labeled.add(seed)
        Q = 0

        while True:
            neighbors = {v for u in C for v in G.neighbors(u) if v not in C}
            if not neighbors:
                break

            contributions = {j: local_contribution(G, C, j) for j in neighbors}
            j_star, q_max = max(contributions.items(), key=lambda item: item[1])

            if q_max >= Q:
                C.add(j_star)
                labeled.add(j_star)
                Q = q_max
            else:
                break

        communities.append(C)
    return communities

def merge_overlapping_communities(communities, threshold=0.7):
    """Stage 2: Merge communities with high overlap."""
    merged = True
    while merged:
        merged = False
        new_communities = []
        used = [False] * len(communities)

        for i, Ci in enumerate(communities):
            if used[i]:
                continue
            merged_comm = set(Ci)
            used[i] = True

            for j in range(i + 1, len(communities)):
                if used[j]:
                    continue
                Cj = communities[j]
                S = len(merged_comm & Cj) / len(merged_comm | Cj)
                if S >= threshold:
                    merged_comm |= Cj
                    used[j] = True
                    merged = True

            new_communities.append(merged_comm)

        communities = new_communities
    return communities

def central_node_overlapping_communities(G, overlap_threshold=0.7):
    """
    Central-node based overlapping community detection.
    Args:
        G: networkx Graph
        overlap_threshold: overlap ratio to merge communities
    Returns:
        List of sets, each a community
    """
    initial_comms = community_mining(G)
    final_comms = merge_overlapping_communities(initial_comms, threshold=overlap_threshold)
    return final_comms


# Test on Karate Club
G = nx.karate_club_graph()
print(f"Karate club graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
comms = central_node_overlapping_communities(G)
print("Detected communities:")
for i, comm in enumerate(comms, 1):
    print(f"Community {i}: {sorted(comm)}")


Karate club graph has 34 nodes and 78 edges.
Detected communities:
Community 1: [2, 8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
Community 2: [0, 1, 2, 3, 7, 8, 9, 11, 12, 13, 17, 19, 21, 30]
Community 3: [4, 5, 6, 10, 16]


### Step 2.2: k‑Faction Algorithm

In [40]:
import networkx as nx
from networkx.algorithms.clique import find_cliques

def k_faction_community_detection(G, k=4, T=0.6, CONN=0.5):
    # Step 1: Find maximal cliques (factions)
    cliques = [set(c) for c in find_cliques(G) if len(c) >= k]

    # Step 2: Merge overlapping cliques into initial communities
    communities = []
    for clique in cliques:
        merged = False
        for i, existing in enumerate(communities):
            overlap = len(clique & existing) / min(len(clique), len(existing))
            if overlap >= T:
                communities[i] = existing | clique
                merged = True
                break
        if not merged:
            communities.append(clique)

    # Step 3: Merge communities based on connectivity
    def interconnectivity(comm1, comm2):
        inter_edges = 0
        total_edges = 0
        for u in comm1:
            for v in comm2:
                if G.has_edge(u, v):
                    inter_edges += 1
        total_edges = len(comm1) * len(comm2)
        return inter_edges / total_edges if total_edges else 0

    merged_flag = True
    while merged_flag:
        merged_flag = False
        new_communities = []
        skip = set()
        for i in range(len(communities)):
            if i in skip:
                continue
            for j in range(i + 1, len(communities)):
                if j in skip:
                    continue
                conn = interconnectivity(communities[i], communities[j])
                if conn >= CONN:
                    merged = communities[i] | communities[j]
                    new_communities.append(merged)
                    skip.update({i, j})
                    merged_flag = True
                    break
            if i not in skip:
                new_communities.append(communities[i])
        communities = new_communities

    # Step 4: Assign remaining nodes
    assigned = set().union(*communities)
    unassigned = set(G.nodes()) - assigned
    for node in unassigned:
        best_comm = None
        max_conn = -1
        for comm in communities:
            conn = sum(1 for neighbor in G.neighbors(node) if neighbor in comm)
            if conn > max_conn:
                max_conn = conn
                best_comm = comm
        if best_comm is not None:
            best_comm.add(node)
        else:
            communities.append({node})  # Isolated node gets its own community

    return communities


### Constructing the user-user network

In [20]:
from collections import defaultdict
import networkx as nx

def build_user_item_bipartite(ratings):
    B = nx.Graph()
    for _, row in ratings.iterrows():
        user = f'u{row["user_id"]}'
        item = f'i{row["item_id"]}'
        B.add_node(user, bipartite=0)
        B.add_node(item, bipartite=1)
        B.add_edge(user, item)
    return B

In [21]:
from networkx.algorithms import bipartite

def project_user_graph(B):
    users = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
    G_user = bipartite.weighted_projected_graph(B, users)
    return G_user

In [38]:
def filter_user_graph(G_user, min_common=5):
    G_filtered = nx.Graph()

    # Add strong edges (based on co-rated items)
    for u, v, data in G_user.edges(data=True):
        if data['weight'] >= min_common:
            G_filtered.add_edge(u, v, weight=data['weight'])

    # Re-add all user nodes (even if isolated)
    G_filtered.add_nodes_from(G_user.nodes(data=True))  # keep attributes if any

    return G_filtered

In [39]:
B = build_user_item_bipartite(ratings)
G_user = project_user_graph(B)
G_filtered = filter_user_graph(G_user, min_common=50)

print(f"Projected user-user graph has {G_filtered.number_of_nodes()} nodes and {G_filtered.number_of_edges()} edges.")

Projected user-user graph has 943 nodes and 41709 edges.


In [84]:
comms = central_node_overlapping_communities(G_filtered)
print("Detected communities:")
for i, comm in enumerate(comms, 1):
    print("Community of size", len(comm))
    print(f"Community {i}: {sorted(comm)}")

Detected communities:
Community of size 57
Community 1: ['u100', 'u106', 'u116', 'u118', 'u121', 'u122', 'u176', 'u195', 'u2', 'u24', 'u243', 'u257', 'u329', 'u331', 'u392', 'u402', 'u421', 'u430', 'u449', 'u451', 'u460', 'u470', 'u480', 'u489', 'u501', 'u526', 'u539', 'u540', 'u569', 'u585', 'u587', 'u655', 'u658', 'u676', 'u683', 'u69', 'u719', 'u724', 'u73', 'u733', 'u734', 'u752', 'u76', 'u768', 'u778', 'u782', 'u79', 'u828', 'u829', 'u831', 'u860', 'u863', 'u877', 'u89', 'u910', 'u931', 'u942']
Community of size 46
Community 2: ['u100', 'u116', 'u118', 'u121', 'u13', 'u195', 'u24', 'u243', 'u255', 'u274', 'u324', 'u329', 'u367', 'u37', 'u392', 'u411', 'u421', 'u451', 'u48', 'u489', 'u546', 'u587', 'u658', 'u676', 'u679', 'u683', 'u724', 'u73', 'u75', 'u752', 'u753', 'u76', 'u771', 'u778', 'u780', 'u782', 'u8', 'u802', 'u828', 'u831', 'u860', 'u863', 'u877', 'u89', 'u942', 'u96']
Community of size 514
Community 3: ['u1', 'u10', 'u100', 'u101', 'u102', 'u104', 'u106', 'u109', 'u11',

In [ ]:
communities = k_faction_community_detection(G_filtered, k=4, T=0.7, CONN=0.7)
print(f"Detected {len(communities)} communities")

for i, comm in enumerate(communities, 1):  # Show first 5
    print("Community of size", len(comm))
    print(f"Community {i}: {sorted(comm)}")

Detected 7 communities
Community 1: ['u1', 'u10', 'u101', 'u102', 'u103', 'u104', 'u105', 'u107', 'u108', 'u109', 'u11', 'u110', 'u111', 'u112', 'u113', 'u114', 'u115', 'u116', 'u119', 'u12', 'u120', 'u121', 'u122', 'u123', 'u124', 'u125', 'u126', 'u127', 'u128', 'u129', 'u13', 'u130', 'u131', 'u132', 'u133', 'u134', 'u135', 'u136', 'u137', 'u138', 'u139', 'u14', 'u140', 'u142', 'u143', 'u144', 'u145', 'u146', 'u147', 'u148', 'u149', 'u15', 'u150', 'u151', 'u152', 'u153', 'u154', 'u155', 'u156', 'u157', 'u158', 'u16', 'u160', 'u161', 'u162', 'u163', 'u164', 'u165', 'u166', 'u167', 'u168', 'u169', 'u17', 'u170', 'u171', 'u172', 'u173', 'u174', 'u175', 'u176', 'u177', 'u178', 'u179', 'u18', 'u180', 'u181', 'u182', 'u183', 'u184', 'u185', 'u187', 'u188', 'u189', 'u19', 'u190', 'u191', 'u192', 'u193', 'u194', 'u196', 'u197', 'u198', 'u199', 'u20', 'u200', 'u201', 'u202', 'u203', 'u204', 'u205', 'u206', 'u207', 'u208', 'u209', 'u21', 'u210', 'u211', 'u212', 'u213', 'u214', 'u215', 'u216', '

## User network based on user tag attributes

In [85]:
import pandas as pd
import networkx as nx

def zip_similarity(zip1, zip2):
    zip1, zip2 = str(zip1), str(zip2)
    return sum(c1 == c2 for c1, c2 in zip(zip1[:3], zip2[:3])) / 3

def demographic_similarity(u1, u2, weights):
    age_sim = max(0, 1 - abs(u1.age - u2.age) / 50)
    gender_sim = 1 if u1.gender == u2.gender else 0
    occupation_sim = 1 if u1.occupation == u2.occupation else 0
    zip_sim = zip_similarity(u1.zip_code, u2.zip_code)

    return (weights['age'] * age_sim +
            weights['gender'] * gender_sim +
            weights['occupation'] * occupation_sim +
            weights['zip'] * zip_sim)

def build_weighted_user_graph(users, weights, threshold=0.5):
    G = nx.Graph()
    for _, row in users.iterrows():
        G.add_node(row.user_id, age=row.age, gender=row.gender,
                   occupation=row.occupation, zip_code=row.zip_code)

    for i, u1 in users.iterrows():
        for j, u2 in users.iterrows():
            if i >= j:
                continue
            sim = demographic_similarity(u1, u2, weights)
            if sim >= threshold:
                G.add_edge(u1.user_id, u2.user_id, weight=sim)
    return G


In [86]:
users = pd.read_csv("Data/ml-100k/u.user", sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

weights = {
    'age': 0.3,
    'gender': 0.2,
    'occupation': 0.3,
    'zip': 0.2
}

G_weighted = build_weighted_user_graph(users, weights, threshold=0.5)

print(f"Weighted user graph: {G_weighted.number_of_nodes()} nodes, {G_weighted.number_of_edges()} edges")

Weighted user graph: 943 nodes, 79161 edges


In [92]:
communities = k_faction_community_detection(G_weighted, k=7, T=0.9, CONN=0.9)
print(f"Detected {len(communities)} communities")

for i, comm in enumerate(communities, 1):  # Show first 5
    print("Community of size", len(comm))
    print(f"Community {i}: {sorted(comm)}")

KeyboardInterrupt: 

## Step 3: Community‑Based Collaborative Filtering

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def compute_combined_similarity(user_item_matrix, user_category_matrix, lambda_=0.5):
    # Ratings-based similarity (cosine)
    sim_r = cosine_similarity(user_item_matrix)
    np.fill_diagonal(sim_r, 0)

    # Category/tag similarity (cosine on binary genre vectors)
    sim_cat = cosine_similarity(user_category_matrix)
    np.fill_diagonal(sim_cat, 0)

    # Final similarity
    combined = (1 - lambda_) * sim_r + lambda_ * sim_cat
    return combined


In [57]:
def build_user_category_matrix(ratings_df, movie_genres_df):
    # movie_genres_df: item_id → binary genre vector
    merged = ratings_df.merge(movie_genres_df, on='item_id')
    genre_cols = movie_genres_df.columns.difference(['item_id'])

    # Aggregate by user
    user_category_matrix = merged.groupby('user_id')[genre_cols].sum()
    user_category_matrix = user_category_matrix.applymap(lambda x: 1 if x > 0 else 0)
    return user_category_matrix


In [58]:
def predict_rating(user_id, item_id, community, user_item_matrix, sim_matrix, k=30):
    if user_id not in community:
        return user_item_matrix.loc[user_id].mean()  # fallback: user average

    neighbors = [u for u in community if u != user_id and user_item_matrix.loc[u, item_id] > 0]
    if not neighbors:
        return user_item_matrix.loc[user_id].mean()

    sims = [(other, sim_matrix[user_id-1, other-1]) for other in neighbors]
    sims = sorted(sims, key=lambda x: x[1], reverse=True)[:k]

    num = sum(sim * user_item_matrix.loc[u, item_id] for u, sim in sims)
    denom = sum(abs(sim) for _, sim in sims)
    return num / denom if denom != 0 else user_item_matrix.loc[user_id].mean()


In [59]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def evaluate(test_df, communities, user_item_matrix, sim_matrix, k=30):
    preds, actuals = [], []

    # Pre-map each user to their community
    user_community = {}
    for c in communities:
        for u in c:
            user_community[int(u[1:])] = [int(n[1:]) for n in c]

    for _, row in test_df.iterrows():
        u, i, r = row['user_id'], row['item_id'], row['rating']
        comm = user_community.get(u, [])
        pred = predict_rating(u, i, comm, user_item_matrix, sim_matrix, k)
        preds.append(pred)
        actuals.append(r)

    mae = mean_absolute_error(actuals, preds)
    rmse = mean_squared_error(actuals, preds, squared=False)
    return mae, rmse


## Testing the approach

In [83]:
communities = k_faction_community_detection(G_filtered, k=7, T=0.9, CONN=0.9)
print(f"Detected {len(communities)} communities")

for i, comm in enumerate(communities, 1):  # Show first 5
    print("Community of size", len(comm))
    print(f"Community {i}: {sorted(comm)}")

Detected 29 communities
Community of size 445
Community 1: ['u103', 'u105', 'u107', 'u108', 'u111', 'u112', 'u113', 'u114', 'u12', 'u120', 'u123', 'u124', 'u126', 'u127', 'u129', 'u13', 'u131', 'u132', 'u133', 'u134', 'u135', 'u136', 'u137', 'u138', 'u139', 'u140', 'u142', 'u143', 'u146', 'u147', 'u149', 'u150', 'u153', 'u154', 'u155', 'u156', 'u157', 'u161', 'u162', 'u163', 'u165', 'u166', 'u167', 'u169', 'u17', 'u170', 'u171', 'u172', 'u173', 'u175', 'u179', 'u182', 'u183', 'u185', 'u19', 'u190', 'u191', 'u192', 'u196', 'u199', 'u20', 'u202', 'u203', 'u204', 'u205', 'u206', 'u208', 'u209', 'u211', 'u212', 'u218', 'u219', 'u220', 'u225', 'u226', 'u227', 'u228', 'u229', 'u231', 'u234', 'u237', 'u238', 'u240', 'u241', 'u242', 'u245', 'u247', 'u252', 'u258', 'u259', 'u260', 'u261', 'u265', 'u266', 'u27', 'u272', 'u273', 'u277', 'u278', 'u281', 'u282', 'u283', 'u284', 'u285', 'u289', 'u29', 'u3', 'u30', 'u300', 'u302', 'u304', 'u306', 'u309', 'u31', 'u310', 'u317', 'u319', 'u32', 'u322', 